# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [6]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [7]:
navegador = webdriver.Chrome("chromedriver.exe")

#Entrar no site desejado (do Google)
navegador.get("https://www.google.com/")

# Passo 1: Pegar a cotação do dólar
# Capturar o elemento da barra de busca do Google (utilizar aspas simples como padrão) e escrever o objeto de busca
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Capturar o valor da cotação
cotacaoDolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacaoDolar)

# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacaoEuro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacaoEuro)

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacaoOuro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacaoOuro = cotacaoOuro.replace(",",".")
print(cotacaoOuro)



5.5327
6.379219698
313.23


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [8]:
# Passo 4: Importar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [9]:
# Passo 5: Atualizar a cotação, preço de compra e o preço de venda

# Atualizar cotação
#variavel.loc[variavel["Coluna"] == "parametro", "coluna"] = TransformarEmDecimal(variavel que armazena os dados de interesse)

tabela.loc[tabela["Moeda"] == "Dólar","Cotação"] = float(cotacaoDolar)
tabela.loc[tabela["Moeda"] == "Euro","Cotação"] = float(cotacaoEuro)
tabela.loc[tabela["Moeda"] == "Ouro","Cotação"] = float(cotacaoOuro)

# Atualizar Preço de Compra = Preço Original * Cotação
tabela["Preço de Compra"] = tabela["Preço Base Original"] * tabela["Cotação"]

# Atualiza Preço de Venda = Preço de Compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R$ {:.2f}".format)

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final,Preço de Compra,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.53270,4999.95,1.40,6999.930,5532.644673,R$ 7745.70
1,Carro Renault,4500.00,Euro,6.37922,27000.00,2.00,54000.000,28706.488641,R$ 57412.98
2,Notebook Dell,899.99,Dólar,5.53270,4499.95,1.70,7649.915,4979.374673,R$ 8464.94
3,IPhone,799.00,Dólar,5.53270,3995.00,1.70,6791.500,4420.627300,R$ 7515.07
4,Carro Fiat,3000.00,Euro,6.37922,18000.00,1.90,34200.000,19137.659094,R$ 36361.55
5,Celular Xiaomi,480.48,Dólar,5.53270,2402.40,2.00,4804.800,2658.351696,R$ 5316.70
6,Joia 20g,20.00,Ouro,313.23000,7000.00,1.15,8050.000,6264.600000,R$ 7204.29


### Agora vamos exportar a nova base de preços atualizada

In [10]:
# Passo 6: Exportar o relatório atualizado

tabela.to_excel("Produtos_Atualizado.xlsx", index=False)
navegador.quit()